In [13]:
import sys; sys.path.append('../')
from DataPreparing.data_preparing import read_data
from Evaluation.evaluate import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


c:\Users\dell\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X_train, X_val, y_train, y_val =read_data("../DataFiles/graphs_representations.csv")

In [3]:
# concatenate x_train and x_val
X_train = np.concatenate((X_train, X_val), axis=0)
y_train = np.concatenate((y_train, y_val), axis=0)


In [4]:
# convert float to float64 
X_train = X_train.astype('float64')
X_val = X_val.astype('float64')


In [5]:
print(set   (y_train))
print(set   (y_val))

{'CWE190', 'CWE23', 'CWE121', 'safe'}
{'CWE190', 'CWE23', 'CWE121', 'safe'}


In [6]:
# encode class values as integers safe=0, CWE23=1
for i in range(len(y_train)):
    if y_train[i]=='safe':
        y_train[i]=0
    elif y_train[i]=='CWE23':
        y_train[i]=1
    elif y_train[i]=='CWE190':
        y_train[i]=2
    elif y_train[i]=='CWE121':
        y_train[i]=3

for i in range(len(y_val)):
    if y_val[i]=='safe':
        y_val[i]=0
    elif y_val[i]=='CWE23':
        y_val[i]=1
    elif y_val[i]=='CWE190':
        y_val[i]=2
    elif y_val[i]=='CWE121':
        y_val[i]=3
    
# for i in range(len(y_val)):
#     if y_val[i]=='safe':
#         y_val[i]=0
#     else:
#         y_val[i]=1

In [7]:
print(set   (y_train))
print(set   (y_val))

{0, 1, 2, 3}
{0, 1, 2, 3}


In [8]:
y_val = y_val.astype('int64')
y_train = y_train.astype('int64')


In [9]:
print(set(y_train))


{0, 1, 2, 3}


In [10]:
#print shape and type of data
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_val shape: ", X_val.shape)
print("y_val shape: ", y_val.shape)


X_train shape:  (16532, 128)
y_train shape:  (16532,)
X_val shape:  (3307, 128)
y_val shape:  (3307,)


In [11]:
print(type(X_train[0][0]))

<class 'numpy.float64'>


In [14]:
X_train_tensor = torch.from_numpy(X_train)
y_train_tensor = torch.from_numpy(y_train)
X_val_tensor = torch.from_numpy(X_val)
y_val_tensor = torch.from_numpy(y_val)


In [15]:
print(X_train_tensor.shape)

torch.Size([16532, 128])


In [21]:

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(128, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Define the dataset
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Prepare the data
data =X_train_tensor
labels =y_train_tensor
dataset = MyDataset(data, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the model, loss function, and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Train the model
for epoch in range(20):
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(dataset)
    epoch_acc = 100 * correct / total
    
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, 10, epoch_loss, epoch_acc))
    #Print classification report
    # print(classification_report(y_val, predicted))

    # epoch_loss = running_loss / len(dataset)
    # print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 10, epoch_loss))

C:\Users\dell\AppData\Local\Temp\ipykernel_17276\3203046694.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(data, dtype=torch.float32)
C:\Users\dell\AppData\Local\Temp\ipykernel_17276\3203046694.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)


Epoch [1/10], Loss: 1.4057, Accuracy: 33.77%
Epoch [2/10], Loss: 1.3823, Accuracy: 36.14%
Epoch [3/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [4/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [5/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [6/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [7/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [8/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [9/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [10/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [11/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [12/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [13/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [14/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [15/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [16/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [17/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [18/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [19/10], Loss: 1.3821, Accuracy: 36.16%
Epoch [20/10], Loss: 1.3821, Accuracy: 36.16%


In [17]:
# import tensorflow as tf
# from tensorflow import keras
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler


# # Define the neural network architecture
# model = keras.Sequential([
#     keras.layers.Dense(32, activation='relu', input_shape=(10,)),
#     keras.layers.Dense(16, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

# # Compile the model with a binary cross-entropy loss function and an Adam optimizer
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train the model on the training data, using early stopping to prevent overfitting
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# model.fit( tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train) , epochs=50, batch_size=32, callbacks=[early_stopping])

# # Evaluate the model on the test data
# loss, accuracy = model.evaluate(X_val, y_val)
# print('Test loss:', loss)
# print('Test accuracy:', accuracy)